In [7]:
import ufl
#import gmsh
import numpy as np
from dolfinx import io,fem,mesh,cpp,plot
from mpi4py import MPI
import pyvista
#gmsh.initialize()

In [8]:
#read in an adcirc mesh and save some stuff
def ADCIRC_mesh_gen(file_path):
    #specify file path as a string, either absolute or relative to where script is run
    #only compatible for adcirc fort.14 format
    adcirc_mesh=open(file_path,'r')
    title=adcirc_mesh.readline()

    #NE number of elements, NP number of grid points
    NE,NP=adcirc_mesh.readline().split()
    NE=int(NE)
    NP=int(NP)

    #initiate data structures
    NODENUM=np.zeros(NP)
    LONS=np.zeros(NP)
    LATS=np.zeros(NP)
    DPS=np.zeros(NP)
    ELEMNUM=np.zeros(NE)
    NM = np.zeros((NE,3)) #stores connectivity at each element

    #read node information line by line 
    for i in range(NP):
        NODENUM[i], LONS[i], LATS[i], DPS[i] = adcirc_mesh.readline().split()
    #read in connectivity
    for i in range(NE):
        ELEMNUM[i], DUM, NM[i,0],NM[i,1], NM[i,2]=adcirc_mesh.readline().split()

    #(we need to shift nodenum down by 1)
    ELEMNUM=ELEMNUM-1
    NM=NM-1
    NODENUM=NODENUM-1

    #close file
    adcirc_mesh.close()
    return NODENUM,[LONS,LATS],ELEMNUM,NM
    

In [9]:
NODENUM,[LONS,LATS],ELEMNUM,NM = ADCIRC_mesh_gen("depth2.grd")
#can read in xdmf mesh file
#filename = 'mesh_test.xdmf'
#encoding= io.XDMFFile.Encoding.HDF5
#with io.XDMFFile(MPI.COMM_WORLD, filename, "r", encoding=encoding) as file:
#    mesh2 = file.read_mesh()

In [10]:
gdim, shape, degree = 2, "triangle", 1
cell = ufl.Cell(shape, geometric_dimension=gdim)
element = ufl.VectorElement("Lagrange", cell, degree)
domain = ufl.Mesh(element)
coords = np.array(list(zip(LONS,LATS)))


#x = np.array([[0., 0., 0.], [0., 1., 0.], [1., 1., 0.]])
#cells = np.array([[0, 1, 2]], dtype=np.int64)


domain1 = mesh.create_mesh(MPI.COMM_WORLD, NM, coords, domain)

V = fem.FunctionSpace(domain1, ("P", 1))
dof_coords = V.tabulate_dof_coordinates()
#assign depth as a function of y
depth = 20 - dof_coords[:,1]/200
f = fem.Function(V)
f.x.array[:] = depth


In [11]:
#want to create a function that approximates h in the P1 space
tdim = domain1.topology.dim
num_cells = domain1.topology.index_map(tdim).size_local
h1 = cpp.mesh.h(domain1, tdim, range(num_cells))
print(h1)

#should be a DG function
cellwise = fem.FunctionSpace(domain1, ("DG", 0))
V = fem.FunctionSpace(domain1,("CG",1))
height = fem.Function(cellwise)
height.vector.setValues(range(num_cells),h1)
#may need to add update ghost for parallel!!!


[28.28427125 28.28427125 27.66651012 ... 23.58013362 22.93498359
 21.41386303]


In [13]:
pyvista.set_jupyter_backend("pythreejs")

# Create plotter and pyvista grid
p = pyvista.Plotter()
topology, cell_types, geometry = plot.create_vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)


plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()


2022-10-12 21:51:24.579 (  98.643s) [         2190000]    vtkExtractEdges.cxx:435   INFO| Executing edge extractor: points are renumbered
2022-10-12 21:51:24.652 (  98.716s) [         2190000]    vtkExtractEdges.cxx:551   INFO| Created 21298 edges


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [12]:
u_topology, u_cell_types, u_geometry = plot.create_vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.cell_data["indicator"] = height.vector.array
u_grid.set_active_scalars("indicator")
pyvista.set_jupyter_backend("pythreejs")
#pyvista.start_xvfb()
p = pyvista.Plotter()
p.add_mesh(u_grid)
p.add_scalar_bar()
p.show_axes()
p.view_xy()
if not pyvista.OFF_SCREEN:
   p.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [15]:
#NOw do L2 interpolation and see what that looks like
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = u*v* ufl.dx
L = height*v*ufl.dx
problem = fem.petsc.LinearProblem(a, L)#petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()



In [16]:
u_topology, u_cell_types, u_geometry = plot.create_vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()

if not pyvista.OFF_SCREEN:
    u_plotter.show()

2022-10-12 21:53:40.657 ( 234.716s) [         2190000]    vtkExtractEdges.cxx:435   INFO| Executing edge extractor: points are renumbered
2022-10-12 21:53:40.730 ( 234.789s) [         2190000]    vtkExtractEdges.cxx:551   INFO| Created 21298 edges


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [ ]:
#plot the mesh and a function on top of it
fname = 'depth_mesh/solution'
xdmf = io.XDMFFile(domain1.comm, fname+".xdmf", "w")
xdmf.write_mesh(domain1)
xdmf.write_function(f)
xdmf.close()